In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
cd drive/MyDrive/Colab\ Notebooks/scripts

/content/drive/MyDrive/Colab Notebooks/scripts


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import sys
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score
import os
from collections import Counter
from imblearn.over_sampling import SMOTE

In [4]:
from feature_selection import forwardFeatureSelection

from GenLinCFA import GenLinCFA

from aux_GenLinCFA import *

In [5]:
def plot_cells(output,selected_colnames, xmin=9, xmax=11, ymin=44, ymax=45.5):
    x = []
    y = []
    colors = cm.rainbow(np.linspace(0,1,len(output)))
    np.random.shuffle(colors)
    fig, ax = plt.subplots(nrows = 5, ncols = 2)
    for j in range(0, 2):
      for i in range(0,5): 
        ax[i][j].set_xlim(xmin,xmax)
        ax[i][j].set_ylim(ymin,ymax)

    subplot_num = 0

    # iterate over all the features aggregations
    for i in range(len(output)):
      # print only the aggregations of the chosen features
      if any(output[i][0][0].split('_')[0].replace("-", "_" ) in feature for feature in selected_colnames_CMI5[i]):
        # j iterates all the clusters
        for j in range (len(output[i])):
            print(len(output[i]))
            x = []
            y = []
            
            for datum in output[i][j]:
                x.append(float(datum.split('_')[1]))
                y.append(float(datum.split('_')[2]))
            ax[subplot_num][0].scatter(x,y,color=colors[j])
            subplot_num += 1
    
    x = []
    y = []
    col = cm.rainbow(np.linspace(0,1,len(selected_colnames)))
    for i in range(0, len(selected_colnames)): 
        idx = int(selected_colnames[i].split('_')[-1])
        for datum in output[idx]:
            x.append(float(datum.split('_')[1]))
            y.append(float(datum.split('_')[2]))
        ax[i][1].scatter(x,y,color=col[i])

# GenLinCFA aggregations original target only on training set



In [ ]:
cd ..

/content/drive/MyDrive/Colab Notebooks


In [ ]:
basins = ['Adda', 'Dora','Emiliani1','Emiliani2', 'Garda_Mincio','Lambro_Olona', 'Oglio_Iseo','Piemonte_Nord','Piemonte_Sud','Ticino']

path_target = './csv/'
path_features = './features_allvalues/'
destination_folder = './GenLinCFA/temp_prec_original_target_04_aggregOnlyTrain/'
target_dest_folder = './GenLinCFA/target_SMOTE_wrapper_04/'
threshold = 0.4

for basin in basins:
    print('####################' + basin + '####################')
    target_df_train, target_df_val, target_df_test, target_df_trainVal = prepare_target_binary(
        '',max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01', 
        path=path_target+basin+'.csv', threshold = threshold, nopeaks = False)
    eps = 0.37
    actual_path = path_features+basin+'_aggreg.csv'
    output, aggregate_train, aggregate_val, aggregate_test, target_train_res = aggregate_unfolded_data_onlyTrain_SMOTE(actual_path,['cyclostationary_mean_tg', 
                                                                             'cyclostationary_mean_tg_1w',
                                                                             'cyclostationary_mean_tg_4w', 
                                                                             'cyclostationary_mean_tg_8w',
                                                                             'cyclostationary_mean_tg_12w', 
                                                                             'cyclostationary_mean_tg_16w',
                                                                             'cyclostationary_mean_tg_24w',
                                                                             'cyclostationary_mean_rr', 
                                                                             'cyclostationary_mean_rr_1w',
                                                                             'cyclostationary_mean_rr_4w', 
                                                                             'cyclostationary_mean_rr_8w',
                                                                             'cyclostationary_mean_rr_12w', 
                                                                             'cyclostationary_mean_rr_16w',
                                                                             'cyclostationary_mean_rr_24w'
                                                                            ],
                                                                       target_df_train, eps=eps,
                                                                       max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01')
    target_train_res = target_train_res.to_frame(name="mean_std")
    target_train_res.to_csv(target_dest_folder + basin + '_train_target_SMOTE.csv')


    aggregate_trainVal = pd.concat([aggregate_train,aggregate_val],axis=0).reset_index(drop=True)
    selected_colnames = FS_with_logisticWrapper(aggregate_trainVal, target_train_res, 
                                                target_df_val, min(50,aggregate_train.shape[1]-1), len(target_df_val))
    target_trainVal_res = pd.concat([target_train_res,target_df_val['mean_std'].to_frame(name="mean_std")],axis=0).reset_index(drop=True)
    print('\nFull model and selected features with wrapper\n')
    compare_methods_class(aggregate_trainVal, aggregate_test, target_trainVal_res, target_df_test, selected_colnames)

    print('\nFull model and best 5 selected features with wrapper\n')
    compare_methods_class(aggregate_trainVal, aggregate_test, target_trainVal_res, target_df_test, selected_colnames[0:5])

    train_string = destination_folder + basin + '_genLinCFA_wrapper_best5_train.csv'
    val_string = destination_folder + basin + '_genLinCFA_wrapper_best5_val.csv'
    test_string = destination_folder + basin + '_genLinCFA_wrapper_best5_test.csv'

    X_train_wrapper = aggregate_train.loc[:, selected_colnames[0:5]]
    X_validation_wrapper = aggregate_val.loc[:, selected_colnames[0:5]]
    X_train_validation_wrapper = pd.concat([X_train_wrapper, X_validation_wrapper])
    X_test_wrapper = aggregate_test.loc[:,selected_colnames[0:5]]
        
    X_train_wrapper.to_csv(train_string, index=False)
    X_validation_wrapper.to_csv(val_string, index=False)
    X_test_wrapper.to_csv(test_string, index=False)

# GenLinCFA results in binary classification

## Features and target - Single Task

In [ ]:
cd ..

/content/drive/MyDrive/Colab Notebooks


In [ ]:
### new targets
basin = "Ticino"
path_orig_targets = './csv/'
path_res_targets = './GenLinCFA/target_SMOTE_wrapper_04/'

threshold = 0.4

targets_df_train = pd.DataFrame()
targets_df_val = pd.DataFrame()
targets_df_test = pd.DataFrame()
targets_df_trainVal = pd.DataFrame()

target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target_binary('',max_train='2010-01-01', max_val='2015-01-01', 
                                                                                        max_test='2020-01-01', path=path_orig_targets+basin+'.csv',
                                                                                        threshold = threshold)
target_df_test = target_df_test['mean_std']
target_df_train = pd.read_csv(path_res_targets + basin + '_train_target_SMOTE.csv')['mean_std']
target_df_val = target_df_val['mean_std']
target_df_trainVal = pd.concat([target_df_train, target_df_val])

target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.264043    0.00  2001     1         0
1    2001-01-13  0.354618    0.39  2001     2         0
2    2001-01-21  0.427990    0.47  2001     3         1
3    2001-01-29  0.339495    0.35  2001     5         0
4    2001-02-06  0.324134    0.34  2001     6         0
..          ...       ...     ...   ...   ...       ...
406  2009-11-27  0.332713    0.35  2009    48         0
407  2009-12-05  0.370253    0.40  2009    49         0
408  2009-12-13  0.517201    0.57  2009    50         1
409  2009-12-21  0.353636    0.45  2009    52         0
410  2009-12-29  0.261079    0.00  2009    53         0

[411 rows x 6 columns]
 target shapes: ((411, 6), (228, 6), (639, 6), (228, 6))


In [ ]:
def get_features (path_features, basin, train_name, val_name, test_name):
  features_train = pd.read_csv(path_features + basin + train_name)
  features_valid = pd.read_csv(path_features + basin + val_name)
  features_test = pd.read_csv(path_features + basin + test_name)
  
  features_train_validation = pd.concat([features_train, features_valid])
  return features_train.values, features_valid.values, features_train_validation.values, features_test.values

In [ ]:
# Temp Prec
path_features = './GenLinCFA/temp_prec_original_target_04_aggregOnlyTrain/'

# wrapper
X_train_wrapper, X_validation_wrapper, X_train_validation_wrapper, X_test_wrapper = get_features(path_features, basin, '_genLinCFA_wrapper_best5_train.csv',
                                                                                                 '_genLinCFA_wrapper_best5_val.csv', '_genLinCFA_wrapper_best5_test.csv')

In [ ]:
log_regr = LogisticRegression(solver='lbfgs', random_state = 42)

# wrapper
log_regr.fit(X_train_validation_wrapper, target_df_trainVal)
print("Train accuracy logistic regression wrapper: ", round(log_regr.score(X_train_validation_wrapper, target_df_trainVal),3))
print("Test accuracy logistic regression wrapper: ", round(log_regr.score(X_test_wrapper, target_df_test),3), "\n")

Train accuracy logistic regression wrapper:  0.655
Test accuracy logistic regression wrapper:  0.662 



## Features and target - Multi Task

In [ ]:
### original targets
basins = ['Adda','Dora','Emiliani1','Emiliani2','Garda_Mincio','Lambro_Olona','Oglio_Iseo','Piemonte_Nord','Piemonte_Sud','Ticino']
path_orig_targets = './csv/'

threshold = 0.5

targets_df_train = pd.DataFrame()
targets_df_val = pd.DataFrame()
targets_df_test = pd.DataFrame()
targets_df_trainVal = pd.DataFrame()

for basin in basins:
    target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target_binary('',max_train='2010-01-01', max_val='2015-01-01', 
                                                                                            max_test='2020-01-01', path=path_orig_targets+basin+'.csv', 
                                                                                            threshold = threshold)
    targets_df_test[basin] = target_df_test['mean_std']
    targets_df_train[basin] = target_df_train['mean_std']
    targets_df_val[basin] = target_df_val['mean_std']
    targets_df_trainVal[basin] = pd.concat([targets_df_train[basin], targets_df_val[basin]])

target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.039373    0.00  2001     1         0
1    2001-01-13  0.380618    0.43  2001     2         0
2    2001-01-21  0.341985    0.38  2001     3         0
3    2001-01-29  0.322044    0.35  2001     5         0
4    2001-02-06  0.354954    0.40  2001     6         0
..          ...       ...     ...   ...   ...       ...
406  2009-11-27  0.382706    0.40  2009    48         0
407  2009-12-05  0.409921    0.46  2009    49         0
408  2009-12-13  0.472087    0.53  2009    50         0
409  2009-12-21  0.324728    0.00  2009    52         0
410  2009-12-29  0.086512    0.00  2009    53         0

[411 rows x 6 columns]
 target shapes: ((411, 6), (228, 6), (639, 6), (228, 6))
target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.010645    0.00  2001     1         0
1    2001-01-13  0.206769    0.00  2001     2         0
2    2001-01-21  0.267313    0.00  2001     3  

#### Temp Prec

In [ ]:
### wrapper best 5 features
path_features = './GenLinCFA/temp_prec_original_target_04_aggregOnlyTrain/'

best5_wrapper_fulldf_train = pd.DataFrame()
best5_wrapper_fulldf_val = pd.DataFrame()
best5_wrapper_fulldf_test = pd.DataFrame()

for basin in basins:
    best5_wrapper_train_temp = pd.read_csv(path_features+basin+'_genLinCFA_wrapper_best5_train.csv')
    best5_wrapper_val_temp = pd.read_csv(path_features+basin+'_genLinCFA_wrapper_best5_val.csv')
    best5_wrapper_train_val_temp = pd.concat((best5_wrapper_train_temp, best5_wrapper_val_temp))
    best5_wrapper_test_temp = pd.read_csv(path_features+basin+'_genLinCFA_wrapper_best5_test.csv')

    best5_wrapper_train_val_temp.reset_index(inplace = True, drop = True)
    best5_wrapper_fulldf_train[basin+'_'+best5_wrapper_train_temp.columns.values] = best5_wrapper_train_val_temp.iloc[:411,:]
    best5_wrapper_fulldf_val[basin+'_'+best5_wrapper_val_temp.columns.values] = best5_wrapper_train_val_temp.iloc[411:639,:]
    best5_wrapper_fulldf_test[basin+'_'+best5_wrapper_test_temp.columns.values] = best5_wrapper_test_temp

best5_wrapper_fulldf_val.reset_index(inplace = True, drop = True)

## Multi Task

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
def MTL_scores_smote(clust_basins, df_train, df_val, df_test, targets_df_train, targets_df_val, targets_df_test):
    
    colnames = [x for x in df_train.columns if x.startswith(tuple(clust_basins))]

    clusterdf_train_withClass = pd.DataFrame()
    clusterdf_val_withClass = pd.DataFrame()
    clusterdf_test_withClass = pd.DataFrame()

    for i in range(len(clust_basins)):
        clusterdf_train_withClass = pd.concat((clusterdf_train_withClass,pd.concat((df_train[colnames],pd.DataFrame(1+i*np.ones(len(df_train)),columns=['basin'])),axis=1)),axis=0)
        clusterdf_val_withClass = pd.concat((clusterdf_val_withClass,pd.concat((df_val[colnames],pd.DataFrame(1+i*np.ones(len(df_val)),columns=['basin'])),axis=1)),axis=0)
        clusterdf_test_withClass = pd.concat((clusterdf_test_withClass,pd.concat((df_test[colnames],pd.DataFrame(1+i*np.ones(len(df_test)),columns=['basin'])),axis=1)),axis=0)
    
    for i in range(len(clust_basins)):
        clusterdf_train_withClass[clust_basins[i]] = clusterdf_train_withClass.apply(lambda x: int(x.basin==i+1),axis=1)
        clusterdf_val_withClass[clust_basins[i]] = clusterdf_val_withClass.apply(lambda x: int(x.basin==i+1),axis=1)
        clusterdf_test_withClass[clust_basins[i]] = clusterdf_test_withClass.apply(lambda x: int(x.basin==i+1),axis=1)

    clusterdf_train_withClass = clusterdf_train_withClass.loc[:,clusterdf_train_withClass.columns != 'basin']
    clusterdf_val_withClass = clusterdf_val_withClass.loc[:,clusterdf_val_withClass.columns != 'basin']
    clusterdf_test_withClass = clusterdf_test_withClass.loc[:,clusterdf_test_withClass.columns != 'basin']

    targets_df_train_unfolded = pd.DataFrame()
    targets_df_val_unfolded = pd.DataFrame()
    targets_df_test_unfolded = pd.DataFrame()
    
    for basin in clust_basins:
        targets_df_train_unfolded =  pd.concat((targets_df_train_unfolded,targets_df_train[basin]),axis=0)
        targets_df_val_unfolded =  pd.concat((targets_df_val_unfolded,targets_df_val[basin]),axis=0)
        targets_df_test_unfolded =  pd.concat((targets_df_test_unfolded,targets_df_test[basin]),axis=0)
    targets_df_train_unfolded = targets_df_train_unfolded.reset_index(drop=True)
    targets_df_val_unfolded = targets_df_val_unfolded.reset_index(drop=True)
    targets_df_test_unfolded = targets_df_test_unfolded.reset_index(drop=True)

    clusterdf_train_val = pd.concat((clusterdf_train_withClass,clusterdf_val_withClass))
    target_df_train_val = pd.concat((targets_df_train_unfolded,targets_df_val_unfolded))

    sm = SMOTE(random_state=42)
    clusterdf_train_val_res, target_df_train_val_res = sm.fit_resample(clusterdf_train_val, target_df_train_val)

    # same scores changing the solver, some differences changing penalty, some improve with l1
    model_ohe = LogisticRegression(max_iter = 500)
    model_ohe.fit(clusterdf_train_val_res.values, target_df_train_val_res.values.ravel())
    
    for basin in clust_basins:
        print(basin)
        res = model_ohe.predict(clusterdf_test_withClass.loc[clusterdf_test_withClass[basin]==1].values)
        print(accuracy_score(targets_df_test[basin].values.ravel(), res))
    #return clusterdf_train_val, clusterdf_train_val_res, target_df_train_val_res

### Wrapper

In [ ]:
MTL_scores_smote(clust_basins=['Emiliani1', 'Emiliani2', 'Garda_Mincio'], df_train=best5_wrapper_fulldf_train, df_val=best5_wrapper_fulldf_val, df_test=best5_wrapper_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Emiliani1
0.6842105263157895
Emiliani2
0.6842105263157895
Garda_Mincio
0.7017543859649122


In [ ]:
MTL_scores_smote(clust_basins=['Adda', 'Lambro_Olona', 'Oglio_Iseo', 'Ticino'], df_train=best5_wrapper_fulldf_train, df_val=best5_wrapper_fulldf_val, df_test=best5_wrapper_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Adda
0.6622807017543859
Lambro_Olona
0.6842105263157895
Oglio_Iseo
0.6140350877192983
Ticino
0.7149122807017544


In [ ]:
MTL_scores_smote(clust_basins=['Dora', 'Piemonte_Nord', 'Piemonte_Sud'], df_train=best5_wrapper_fulldf_train, df_val=best5_wrapper_fulldf_val, df_test=best5_wrapper_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Dora
0.7324561403508771
Piemonte_Nord
0.6096491228070176
Piemonte_Sud
0.6228070175438597


In [ ]:
MTL_scores_smote(clust_basins=['Piemonte_Nord', 'Piemonte_Sud'], df_train=best5_wrapper_fulldf_train, df_val=best5_wrapper_fulldf_val, df_test=best5_wrapper_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Piemonte_Nord
0.618421052631579
Piemonte_Sud
0.6008771929824561
